# SITCOM - 702

Repeat of SITCOM-627, combining Ioana's notebook and Guillem's AOS notebook.

This notebook contains the execution for the measuring of astigmatism described in https://jira.lsstcorp.org/browse/SITCOM-627.

The notebook is organized in 6 sections:

1. Conditions assessment
2. Setup notebook
3. Declare target and filter
4. Slew to target object.
5. Check target star field, signal level and declare exposure time. 
6. Perform the data acquisition sequence. 



Goal: Directly measure image degradation due to astigmatism on AuxTel by sweeping through focus.

Idea: If we sweep through focus we will be able to measure image degradation due to astigmatism by (a) comparing the FWHM of each individual axis at its best focus to the FWHM of the circle of least confusion and (b) measuring the distance between the best focus of each axis (these measurements should agree, and hopefully also agree with Zernikes).

Observing sequence (must occur on a night of decent seeing (<1.5 arcsec):

Slew to a nearby bright star and perform CWFS/WEP.
Focus on star of magnitude 7-10, with elevation between 45 and 75 degrees, with the target star in a position where spectroscopy can be performed.

    - Before 3am CLT, use HD60753. 
    - Between 3 and 4am CLT, use HD115169.
    
Sweep through focus by moving M2 in steps of 0.02 mm from -0.1 mm to +0.1 mm (the idea is to move the focal plane in steps of ~1mm). At each position:

    - Take 2x 30s exposures at each position with the red filter in.
    - Take 2x 30s exposures at each position with the holo disperser in and no blocking filters.
    
    
# Assess that conditions meet criteria

This test should be performed when the seeing is "decent", under 1 arcsec.

Preferably, run this execution later in the night, when dome seeing has settled down and seeing is more stable. Confirm that the temperatures in and outside the dome are within 1 degree.



## Questions I still need to answer:

1. Does the holo disperser automatically align itself? - Answer from Erik: yes
2. Where are the scripts that I am running/where do I need to run this notebook from?
3. What targets do I want to use?
4. Does this script run successfully on TTS?

# Setup

### Import Libraries

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_wep_align import LatissWEPAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

import yaml
import logging
import jsonschema
import asyncio

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel import ATQueue
from lsst.ts.idl.enums.ScriptQueue import Location, SalIndex
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.utils import RotType

### Setting up the logger

In [ ]:
logger = logging.getLogger("SITCOM-702")
logger.level = logging.DEBUG

### Getting unique index for script

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

### Instantiate CWFS Script

In [ ]:
script = LatissWEPAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [ ]:
atcs = script.atcs
latiss = script.latiss

### Set up script log level

In [ ]:
script.log.level = logging.DEBUG

### Write start info into EFD

In [ ]:
script.log.info(f'START -- SITCOM-702 Astigmatism test -- at {Time.now()}')

# Target and filter

### Declare target: Choose target.

Based on the elevation vs time plot available in the jira ticket https://jira.lsstcorp.org/browse/SITCOM-702, choose the target that is between 45 and 75 degrees elevation at the time of executing this notebook.

In [ ]:
target = 'HD 60753' # or whatever
logger.info(f'Selected target is {target}')

# Slew to target

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

### ATAOS corrections

ATAOS corrections must be enabled for this test. In the cell below, ATAOS corrections will be enabled, in case they were not.

In [ ]:
corrections_enabled = await script.atcs.rem.ataos.evt_correctionEnabled.aget()
if not (corrections_enabled.m1 and corrections_enabled.hexapod and corrections_enabled.atspectrograph):
    cmd = await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)
    logger.info(f'ATAOS corrections enabled: {cmd.result}')
else:
    logger.info(f'ATAOS corrections already enabled')

# Taking measurements

## PERFORM WEP AND THEN CONTINUE

In [ ]:
offsets = {
    "z": np.linspace(-0.1,0.1, 11),
}


In [ ]:
script_path = "auxtel/latiss_intra_extra_focal_data.py"
band_filter = 'r'
exposure_time = 30

for var, offset_list in offsets.items():
    print(var)
    for offset in offset_list:
        script_config = {
            'filter': f"SDSS{band_filter}_65mm", 
            'grating': 'empty_1',
            'exposure_time': exposure_time,
            f'offset_{var}': float(offset),
            'reason': f'Focus_Sweep_offset_{var}_{offset:.3f}',
            'program': 'SITCOM-702',
        }
        script_description=f"Testing SITCOM-702 offset_{var} {offset}"

        await atq.validate_config(is_standard=False, script=script_path, config=script_config)

        await atq.add_external(script=script_path, config=script_config, description=script_description)


In [ ]:
script_path = "auxtel/latiss_intra_extra_focal_data.py"
exposure_time = 30

for var, offset_list in offsets.items():
    print(var)
    for offset in offset_list:
        script_config = {
            'filter': "empty_1", 
            'grating': 'holo4_003',
            'exposure_time': exposure_time,
            f'offset_{var}': float(offset),
            'reason': f'Focus_Sweep_offset_{var}_{offset:.3f}',
            'program': 'SITCOM-702',
        }
        script_description=f"Testing SITCOM-702 offset_{var} {offset}"

        await atq.validate_config(is_standard=False, script=script_path, config=script_config)

        await atq.add_external(script=script_path, config=script_config, description=script_description)


In [ ]:
# Move back to the original focus offset position
await atcs.rem.ataos.cmd_offset.set_start(z= -(z_offset_end + z_offset_step))
logger.info(f'Back to the original focus offset position \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')

In [ ]:
# Check that it's back in the original focus position
current_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
script.log.info(f'Current focus offset is {current_focus_offset}'

In [ ]:
diff = current_focus_offset.userApplied - original_focus_offset.userApplied
print(f'Difference between current and original user Applied focus is {diff:0.2f} mm')

## Write end info into EFD

In [ ]:
script.log.info(f'END -- SITCOM-702 Astigmatism test -- at {Time.now()}')